In [ ]:
import pandas as pd
import numpy as np
import re
import copy
from datetime import datetime

# Новый раздел

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
pharm_df = pd.read_excel("/content/drive/My Drive/SHAD/Pharm/Universe coded OK July20.xlsx")

In [ ]:
pharm_df.tail()

,OneKeyID,GlobalCode,TetCode,BrandCodes,OKBrickL3,AuditBrickL4,BrickCode,SquareMeterPrice,FlatArea,Metro,BeltwayHit,RegionType,CityType,SettlementType,StreetType,Cash,Card,Internet,Bank,Mon,Tue,Wed,Thu,Fri,Sat,Sun,population,VRP,NumHos,NumAmb,NumPhc,UNIQ_SKU_Jun19,UNIQ_SKU_Jul19,UNIQ_SKU_Aug19,UNIQ_SKU_Sep19,UNIQ_SKU_Oct19,UNIQ_SKU_Nov19,UNIQ_SKU_Dec19,UNIQ_SKU_Jan20,UNIQ_SKU_Feb20,UNIQ_SKU_Mar20,UNIQ_SKU_Apr20,UNIQ_SKU_May20,SELL_Jun19,SELL_Jul19,SELL_Aug19,SELL_Sep19,SELL_Oct19,SELL_Nov19,SELL_Dec19,SELL_Jan20,SELL_Feb20,SELL_Mar20,SELL_Apr20,SELL_May20,Basket1,Basket2,Basket3
65172,WRUH00051711,NaN,PHK,0,"Курская обл, Курск г, Центральный округ","Курская область, Курск",800128,NaN,NaN,NaN,NaN,обл,г,NaN,ул,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,223155.0,346340,35.0,45.0,36.0,2.0,3.0,0.0,5.0,3.0,1.0,3.0,0.0,4.0,2.0,1.0,5.0,7.0,6.0,0.0,16.0,8.0,10.0,5.0,0.0,9.0,20.0,30.0,13.0,NaN,NaN,NaN
65173,WRUH00053774,NaN,PHK,0,"Оренбургская обл, Сакмарский р-н, Сакмара с","Оренбургская область, Прочее",800180,NaN,28.00,NaN,NaN,обл,NaN,с,ул,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4999.0,414937,8.0,2.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65174,WRUH00070966,NaN,PHN,84,"Челябинская обл, Челябинск г, Калининский округ","Челябинская область, Челябинск",800107,NaN,NaN,NaN,NaN,обл,г,NaN,ул,1.0,1.0,NaN,NaN,Понедельник: с 9:00 до 20:00,Вторник: с 9:00 до 20:00,Среда: с 9:00 до 20:00,Четверг: с 9:00 до 20:00,Пятница: с 9:00 до 20:00,Суббота: с 9:00 до 20:00,Воскресенье: с 9:00 до 20:00,226227.0,385560,1.0,50.0,33.0,125.0,12.0,0.0,110.0,105.0,106.0,107.0,106.0,100.0,97.0,97.0,87.0,697.0,30.0,0.0,406.0,359.0,367.0,407.0,436.0,390.0,422.0,435.0,331.0,NaN,NaN,NaN
65175,WRUH00072455,NaN,PHN,163,"Краснодарский край, Городской округ г Краснода...","Краснодарский край, Краснодар",800122,46131.0,NaN,NaN,NaN,край,г,NaN,ул,1.0,1.0,NaN,NaN,Понедельник: с 8:00 до 21:00,Вторник: с 8:00 до 21:00,Среда: с 8:00 до 21:00,Четверг: с 8:00 до 21:00,Пятница: с 8:00 до 21:00,Суббота: с 8:00 до 21:00,Воскресенье: с 8:00 до 21:00,364980.0,398397,112.0,71.0,72.0,116.0,110.0,29.0,98.0,96.0,100.0,105.0,101.0,99.0,101.0,83.0,79.0,1277.0,960.0,85.0,454.0,534.0,643.0,943.0,545.0,634.0,647.0,550.0,596.0,NaN,NaN,NaN
65176,WRUH00075341,NaN,PHN,0,"Москва г, Троицкий округ, Троицк городской округ","Москва, Новомосковский АО & Троицкий АО",800275,NaN,NaN,NaN,OUT_MKAD,г,г,NaN,ул,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61079.0,1263698,0.0,0.0,0.0,64.0,71.0,62.0,31.0,61.0,36.0,27.0,37.0,29.0,28.0,36.0,47.0,382.0,345.0,345.0,113.0,233.0,161.0,189.0,163.0,116.0,197.0,179.0,234.0,NaN,NaN,NaN


# **Удаление данных о Москве, Петербурге, Московской области**

In [ ]:
pharm_df.drop(index= np.concatenate((np.where(pharm_df["OKBrickL3"].map(lambda x: re.findall("петербург",x.lower())).map(lambda x: len(x)) != 0)[0],
                                     np.where(pharm_df["OKBrickL3"].map(lambda x: re.findall("москва",x.lower())).map(lambda x: len(x)) != 0)[0])),
              inplace=True)

In [ ]:
pharm_df.shape

(58731, 58)

In [ ]:
pharm_df.drop(index = pharm_df.iloc[np.where(pharm_df["OKBrickL3"].map(lambda x: re.findall("московская обл",x.lower())).map(lambda x: len(x)) != 0)[0]].index,
              inplace =True)

In [ ]:
pharm_df.shape

(54526, 58)

# **One-hot encoding для типа аптеки**

In [ ]:
pd.get_dummies(pharm_df.TetCode)

,APT,PCP,PHK,PHN,PST
0,0,0,1,0,0
1,0,1,0,0,0
2,1,0,0,0,0
3,0,0,1,0,0
4,1,0,0,0,0
...,...,...,...,...,...
65171,0,0,0,1,0
65172,0,0,1,0,0
65173,0,0,1,0,0
65174,0,0,0,1,0


In [ ]:
pharm_df = pd.concat([pharm_df, pd.get_dummies(pharm_df.TetCode)],axis=1).drop(['TetCode'],axis=1)

# **One-hot для топ N брендов аптечных сетей (признак BrandCode) по объему и по популярности**

In [ ]:
pharm_df.BrandCodes.value_counts()

0      14782
1       7051
11      1922
7       1829
14      1741
       ...  
473        1
306        1
185        1
852        1
809        1
Name: BrandCodes, Length: 851, dtype: int64

In [ ]:
counts = pharm_df.BrandCodes.value_counts()
repl = counts[counts <= counts.iloc[10]].index
top_brand_df = pd.get_dummies(pharm_df.BrandCodes.replace(repl, 'uncommon')).drop(['uncommon'], axis=1)
top_brand_df.rename(columns={x:"top_brand_" + str(x) for x in top_brand_df.columns},inplace=True)
pharm_df = pd.concat([pharm_df, top_brand_df], axis=1)

По объему


In [ ]:
pharm_df.head()

,OneKeyID,GlobalCode,BrandCodes,OKBrickL3,AuditBrickL4,BrickCode,SquareMeterPrice,FlatArea,Metro,BeltwayHit,RegionType,CityType,SettlementType,StreetType,Cash,Card,Internet,Bank,Mon,Tue,Wed,Thu,Fri,Sat,Sun,population,VRP,NumHos,NumAmb,NumPhc,UNIQ_SKU_Jun19,UNIQ_SKU_Jul19,UNIQ_SKU_Aug19,UNIQ_SKU_Sep19,UNIQ_SKU_Oct19,UNIQ_SKU_Nov19,UNIQ_SKU_Dec19,UNIQ_SKU_Jan20,UNIQ_SKU_Feb20,UNIQ_SKU_Mar20,UNIQ_SKU_Apr20,UNIQ_SKU_May20,SELL_Jun19,SELL_Jul19,SELL_Aug19,SELL_Sep19,SELL_Oct19,SELL_Nov19,SELL_Dec19,SELL_Jan20,SELL_Feb20,SELL_Mar20,SELL_Apr20,SELL_May20,Basket1,Basket2,Basket3,APT,PCP,PHK,PHN,PST,top_brand_0,top_brand_1,top_brand_2,top_brand_7,top_brand_10,top_brand_11,top_brand_14,top_brand_17,top_brand_31,top_brand_84
0,WRUE00003190,NaN,0,"Брянская обл, Брянск г, Советский округ","Брянская область, Брянск",800280,NaN,NaN,NaN,NaN,обл,г,NaN,ул,1.0,1.0,NaN,NaN,Понедельник: с 8:00 до 18:00,Вторник: с 8:00 до 18:00,Среда: с 8:00 до 18:00,Четверг: с 8:00 до 18:00,Пятница: с 8:00 до 18:00,Суббота: с 9:00 до 14:00,Воскресенье: -,112158.0,253100,16.0,27.0,12.0,20.0,15.0,8.0,18.0,24.0,12.0,18.0,13.0,16.0,26.0,19.0,13.0,137.0,64.0,67.0,134.0,100.0,69.0,206.0,53.0,101.0,201.0,175.0,83.0,NaN,NaN,NaN,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
1,WRUE00011494,2052648.0,1,"Челябинская обл, Челябинск г, Курчатовский округ","Челябинская область, Челябинск",800107,NaN,NaN,NaN,NaN,обл,г,NaN,пр-кт,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,227841.0,385560,50.0,86.0,37.0,4.0,1.0,0.0,15.0,17.0,19.0,9.0,3.0,3.0,3.0,3.0,3.0,4.0,1.0,0.0,93.0,75.0,89.0,55.0,3.0,3.0,6.0,3.0,3.0,NaN,NaN,NaN,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
2,WRUE00016734,NaN,0,"Краснодарский край, Городской округ Новороссий...","Краснодарский край, Новороссийск",800306,NaN,438.00,NaN,NaN,край,г,NaN,ул,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,147609.0,398397,47.0,31.0,31.0,17.0,16.0,15.0,0.0,0.0,0.0,0.0,0.0,8.0,21.0,25.0,16.0,51.0,34.0,41.0,0.0,0.0,0.0,0.0,0.0,36.0,73.0,100.0,51.0,NaN,NaN,NaN,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,WRUE00016882,NaN,0,"Новгородская обл, Великий Новгород г, Великий ...","Новгородская область, Великий Новгород",800164,NaN,NaN,NaN,NaN,обл,г,NaN,пр-кт,1.0,1.0,NaN,NaN,Понедельник: с 8:00 до 19:00,Вторник: с 8:00 до 19:00,Среда: с 8:00 до 19:00,Четверг: с 8:00 до 19:00,Пятница: с 8:00 до 19:00,Суббота: -,Воскресенье: -,224297.0,441932,37.0,39.0,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
4,WRUE00020818,NaN,0,"Ростовская обл, Ростов-на-Дону г, Октябрьский ...","Ростовская область, Ростов-на-Дону, Октябрьски...",800200,NaN,NaN,NaN,NaN,обл,г,NaN,ул,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,167935.0,318782,0.0,21.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [ ]:
counts = pharm_df.groupby("BrandCodes")[pharm_df.loc[:,"SELL_Jun19":"SELL_May20"].columns].agg(sum).sum(axis=1).sort_values(ascending=False)

In [ ]:
counts

BrandCodes
1      5.426846e+07
0      4.658685e+07
11     1.942230e+07
31     1.922467e+07
14     1.767819e+07
           ...     
888    0.000000e+00
508    0.000000e+00
824    0.000000e+00
830    0.000000e+00
792    0.000000e+00
Length: 851, dtype: float64

In [ ]:
repl = counts[counts <= counts.iloc[15]].index
top_sales_df = pd.get_dummies(pharm_df.BrandCodes.replace(repl, 'uncommon')).drop(['uncommon'], axis=1)
top_sales_df.rename(columns={x:"top_sales_" + str(x) for x in top_sales_df.columns},inplace=True)
pharm_df  = pd.concat([pharm_df, top_sales_df], axis=1)

# **Парсиниг минимального растояния до метро и количества станций метро рядом**

In [ ]:
pharm_df["Metro_dist"] = pharm_df.Metro.map(lambda x: min(re.findall("\d.\d+",x)) if not pd.isna(x) else -1)
pharm_df["Metro_num"] = pharm_df.Metro.map(lambda x: len(re.findall("\d.\d+",x)) if not pd.isna(x) else 0)
pharm_df.drop(["Metro"],axis=1,inplace=True)

# **Парсинг часов и дней работы**

In [ ]:
def get_work_hours(work_time_list):
    '''
    Определение суммарного времени работы аптеки в часах
    * work_time_list - список времен открытия и закрытия (времена в виде строк)
    '''
    # преобразование строк в формат времени
    times = [datetime.strptime(time, "%H:%M") 
             if time != "24:00" and time != "-" else datetime.strptime("0:00", "%H:%M") 
             for time in work_time_list]
    # времена работы до обеда и после обеда
    delta_time_1, delta_time_2 = 0, 0
    if len(times) == 4:
        delta_time_2 = (times[3] - times[2]).seconds  # время работы после обеда
        if delta_time_2 == 0:
            delta_time_2 = 24 * 3600
    if len(times) == 1:  # прочерк, нерабочий день
        return 0
    elif len(times) != 0:
        delta_time_1 = (times[1] - times[0]).seconds  # время работы до обеда
        if delta_time_1 == 0:
            delta_time_1 = 24 * 3600
    else:
        return np.nan
    # суммарное время работы в часах
    work_hours = (delta_time_1 + delta_time_2) / 3600
    return work_hours

In [ ]:
# найдем числа, а также пустые строки и тире для нерабочих дней
work_hours_mon = pharm_df.Mon.map(lambda x: re.findall("\d+:\d+|-", x) if not pd.isnull(x) else "")
work_hours_tue = pharm_df.Tue.map(lambda x: re.findall("\d+:\d+|-", x) if not pd.isnull(x) else "")
work_hours_wed = pharm_df.Wed.map(lambda x: re.findall("\d+:\d+|-", x) if not pd.isnull(x) else "")
work_hours_thu = pharm_df.Thu.map(lambda x: re.findall("\d+:\d+|-", x) if not pd.isnull(x) else "")
work_hours_fri = pharm_df.Fri.map(lambda x: re.findall("\d+:\d+|-", x) if not pd.isnull(x) else "")
work_hours_sat = pharm_df.Sat.map(lambda x: re.findall("\d+:\d{0,2}|-", x) if not pd.isnull(x) else "")
work_hours_sun_ = pharm_df.Sun.map(lambda x: "Воскресенье:-" if x == "Воскресенье:" else x)
work_hours_sun = work_hours_sun_.map(lambda x: re.findall("\d+:\d+|-", x) if not pd.isnull(x) else "")

Есть такие дни, когда в воскресение в поле "рабочие часы" стоит Null. При этом другие дни рабочие. В этом случае время работы в субботу почти в 100% случаев "битое" после обеда. Поэтому после обеда в субботу берем время работы такое же, как в пятницу после обеда.

Если проанализировать первые цифры в "битом" времени, то они совпадают. Значит будем считать, что гипотеза по поводу времени работы в субботу после обеда верна.

Время работы в воскресение возьмем таким же, как в субботу.

In [ ]:
for i in work_hours_sun.index:
    # если время работы в пн указано, а в воскресение Null
    # при этом в пт также есть обед (будем время с пятницы копировать)
    if work_hours_sun[i] == '' and work_hours_mon[i] != '' and len(work_hours_fri[i]) == 4:
        if len(work_hours_sat) == 2:
            work_hours_sat.append(work_hours_fri[2])
            work_hours_sat.append(work_hours_fri[3])
        elif len(work_hours_sat) == 3:
            work_hours_sat[2] = work_hours_fri[2]
            work_hours_sat.append(work_hours_fri[3])
        else:
            work_hours_sat[2] = work_hours_fri[2]
            work_hours_sat[3] = work_hours_fri[3]
        # возьмем время работы в вск такое же, как в субботу
        work_hours_sun[i] = copy.copy(work_hours_sat[i])     

In [ ]:
# суммарные времена работы аптек по дням
mon_work_time = work_hours_mon.apply(get_work_hours)  # понедельник
tue_work_time = work_hours_tue.apply(get_work_hours)  # вторник
wed_work_time = work_hours_wed.apply(get_work_hours)  # средка
thu_work_time = work_hours_thu.apply(get_work_hours)  # четверг
fri_work_time = work_hours_fri.apply(get_work_hours)  # пятница
sat_work_time = work_hours_sat.apply(get_work_hours)  # суббота
sun_work_time = work_hours_sun.apply(get_work_hours)  # воскресение

In [ ]:
# суммарное время работы в будни
weekdays_work_time = mon_work_time + tue_work_time + wed_work_time + \
                    thu_work_time + fri_work_time

In [ ]:
# суммарное время работы в выходные
weekend_work_time = sat_work_time + sun_work_time

In [ ]:
# суммарное время работы за неделю
all_days_work_time = weekdays_work_time + weekend_work_time

In [ ]:
# признак, работает ли аптека 24 на 7 или нет
# финт ушами с копированием, чтобы Null учесть
is_work_time_24_7 = copy.copy(all_days_work_time)
for i in is_work_time_24_7.index:
    if is_work_time_24_7[i] == 24 * 7:
        is_work_time_24_7[i] = 1
    elif not pd.isnull(is_work_time_24_7[i]):
        is_work_time_24_7[i] = 0

In [ ]:
# еще добавим признак, если аптека работает почти 24 на 7
# потому что есть аптеки, которые делают небольшой перерыв ночью
# вряд ли он сильно на деятельности сказывается
# возьмем в среднем 23 на 7
is_work_time_almost_24_7 = copy.copy(all_days_work_time)
for i in is_work_time_almost_24_7.index:
    if is_work_time_almost_24_7[i] != "" and is_work_time_almost_24_7[i] >= 23 * 7:
        is_work_time_almost_24_7[i] = 1
    elif not pd.isnull(is_work_time_almost_24_7[i]):
        is_work_time_almost_24_7[i] = 0

Порядка 50 аптек добавилось. При этом число не меняется, если 22 на 7 ставить. Вероятно, такие аптеки тоже к круглосуточным можно отнести. Просто они делают технический перерыв.

In [ ]:
# добавление признаков в таблицу
pharm_df["mon_work_time"] = mon_work_time
pharm_df["tue_work_time"] = tue_work_time
pharm_df["wed_work_time"] = wed_work_time
pharm_df["thu_work_time"] = thu_work_time
pharm_df["fri_work_time"] = fri_work_time
pharm_df["sat_work_time"] = sat_work_time
pharm_df["sun_work_time"] = sun_work_time

pharm_df["weekdays_work_time"] = weekdays_work_time
pharm_df["weekend_work_time"] = weekend_work_time
pharm_df["all_days_work_time"] = all_days_work_time

pharm_df["is_work_time_24_7"] = is_work_time_24_7
pharm_df["is_work_time_almost_24_7"] = is_work_time_almost_24_7

In [ ]:
pharm_df.head()

,OneKeyID,GlobalCode,BrandCodes,OKBrickL3,AuditBrickL4,BrickCode,SquareMeterPrice,FlatArea,BeltwayHit,RegionType,CityType,SettlementType,StreetType,Cash,Card,Internet,Bank,Mon,Tue,Wed,Thu,Fri,Sat,Sun,population,VRP,NumHos,NumAmb,NumPhc,UNIQ_SKU_Jun19,UNIQ_SKU_Jul19,UNIQ_SKU_Aug19,UNIQ_SKU_Sep19,UNIQ_SKU_Oct19,UNIQ_SKU_Nov19,UNIQ_SKU_Dec19,UNIQ_SKU_Jan20,UNIQ_SKU_Feb20,UNIQ_SKU_Mar20,UNIQ_SKU_Apr20,...,PST,top_brand_0,top_brand_1,top_brand_2,top_brand_7,top_brand_10,top_brand_11,top_brand_14,top_brand_17,top_brand_31,top_brand_84,top_sales_0,top_sales_1,top_sales_7,top_sales_9,top_sales_10,top_sales_11,top_sales_14,top_sales_17,top_sales_31,top_sales_32,top_sales_34,top_sales_39,top_sales_41,top_sales_84,top_sales_133,Metro_dist,Metro_num,mon_work_time,tue_work_time,wed_work_time,thu_work_time,fri_work_time,sat_work_time,sun_work_time,weekdays_work_time,weekend_work_time,all_days_work_time,is_work_time_24_7,is_work_time_almost_24_7
0,WRUE00003190,NaN,0,"Брянская обл, Брянск г, Советский округ","Брянская область, Брянск",800280,NaN,NaN,NaN,обл,г,NaN,ул,1.0,1.0,NaN,NaN,Понедельник: с 8:00 до 18:00,Вторник: с 8:00 до 18:00,Среда: с 8:00 до 18:00,Четверг: с 8:00 до 18:00,Пятница: с 8:00 до 18:00,Суббота: с 9:00 до 14:00,Воскресенье: -,112158.0,253100,16.0,27.0,12.0,20.0,15.0,8.0,18.0,24.0,12.0,18.0,13.0,16.0,26.0,19.0,...,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,10.0,10.0,10.0,10.0,10.0,5.0,0.0,50.0,5.0,55.0,0.0,0.0
1,WRUE00011494,2052648.0,1,"Челябинская обл, Челябинск г, Курчатовский округ","Челябинская область, Челябинск",800107,NaN,NaN,NaN,обл,г,NaN,пр-кт,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,227841.0,385560,50.0,86.0,37.0,4.0,1.0,0.0,15.0,17.0,19.0,9.0,3.0,3.0,3.0,3.0,...,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,WRUE00016734,NaN,0,"Краснодарский край, Городской округ Новороссий...","Краснодарский край, Новороссийск",800306,NaN,438.00,NaN,край,г,NaN,ул,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,147609.0,398397,47.0,31.0,31.0,17.0,16.0,15.0,0.0,0.0,0.0,0.0,0.0,8.0,21.0,25.0,...,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,WRUE00016882,NaN,0,"Новгородская обл, Великий Новгород г, Великий ...","Новгородская область, Великий Новгород",800164,NaN,NaN,NaN,обл,г,NaN,пр-кт,1.0,1.0,NaN,NaN,Понедельник: с 8:00 до 19:00,Вторник: с 8:00 до 19:00,Среда: с 8:00 до 19:00,Четверг: с 8:00 до 19:00,Пятница: с 8:00 до 19:00,Суббота: -,Воскресенье: -,224297.0,441932,37.0,39.0,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,11.0,11.0,11.0,11.0,11.0,11.0,0.0,55.0,11.0,66.0,0.0,0.0
4,WRUE00020818,NaN,0,"Ростовская обл, Ростов-на-Дону г, Октябрьский ...","Ростовская область, Ростов-на-Дону, Октябрьски...",800200,NaN,NaN,NaN,обл,г,NaN,ул,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,167935.0,318782,0.0,21.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# **Агрегация UNIQ_SKU_ & Sell_**

## *UNIQ_SKU_*

In [ ]:
# названия столбцов с уникальными препаратами
uniq_sku_col = [col for col in pharm_df.columns if "UNIQ_SKU_" in col]
# названия столбцов с количеством проданных уникальных препаратов 
sell_col = [col for col in pharm_df.columns if "SELL_" in col]

In [ ]:
# сумма по всем месяцам уникальных препаратов
uniq_sku_sum = pharm_df[uniq_sku_col].sum(axis=1, skipna=False)

In [ ]:
# сделаем сумму еще по сезонам (лето, осень, зима, весна)
uniq_sku_sum_summer = pharm_df[uniq_sku_col[0:3]].sum(axis=1, skipna=False)  # лето
uniq_sku_sum_autumn = pharm_df[uniq_sku_col[3:6]].sum(axis=1, skipna=False)  # осень
uniq_sku_sum_winter = pharm_df[uniq_sku_col[6:9]].sum(axis=1, skipna=False)  # зима
uniq_sku_sum_spring = pharm_df[uniq_sku_col[9:12]].sum(axis=1, skipna=False)  # весна

In [ ]:
# среднее по всем месяцам уникальных препаратов
uniq_sku_mean = pharm_df[uniq_sku_col].mean(axis=1, skipna=False)

In [ ]:
# сделаем среднее еще по сезонам (лето, осень, зима, весна)
uniq_sku_mean_summer = pharm_df[uniq_sku_col[0:3]].mean(axis=1, skipna=False)  # лето
uniq_sku_mean_autumn = pharm_df[uniq_sku_col[3:6]].mean(axis=1, skipna=False)  # осень
uniq_sku_mean_winter = pharm_df[uniq_sku_col[6:9]].mean(axis=1, skipna=False)  # зима
uniq_sku_mean_spring = pharm_df[uniq_sku_col[9:12]].mean(axis=1, skipna=False)  # весна

In [ ]:
# добавление признаков в таблицу
pharm_df["uniq_sku_sum"] = uniq_sku_sum
pharm_df["uniq_sku_sum_summer"] = uniq_sku_sum_summer
pharm_df["uniq_sku_sum_autumn"] = uniq_sku_sum_autumn
pharm_df["uniq_sku_sum_winter"] = uniq_sku_sum_winter
pharm_df["uniq_sku_sum_spring"] = uniq_sku_sum_spring

pharm_df["uniq_sku_mean"] = uniq_sku_mean
pharm_df["uniq_sku_mean_summer"] = uniq_sku_mean_summer
pharm_df["uniq_sku_mean_autumn"] = uniq_sku_mean_autumn
pharm_df["uniq_sku_mean_winter"] = uniq_sku_mean_winter
pharm_df["uniq_sku_mean_spring"] = uniq_sku_mean_spring

In [ ]:
pharm_df.head()

,OneKeyID,GlobalCode,BrandCodes,OKBrickL3,AuditBrickL4,BrickCode,SquareMeterPrice,FlatArea,BeltwayHit,RegionType,CityType,SettlementType,StreetType,Cash,Card,Internet,Bank,Mon,Tue,Wed,Thu,Fri,Sat,Sun,population,VRP,NumHos,NumAmb,NumPhc,UNIQ_SKU_Jun19,UNIQ_SKU_Jul19,UNIQ_SKU_Aug19,UNIQ_SKU_Sep19,UNIQ_SKU_Oct19,UNIQ_SKU_Nov19,UNIQ_SKU_Dec19,UNIQ_SKU_Jan20,UNIQ_SKU_Feb20,UNIQ_SKU_Mar20,UNIQ_SKU_Apr20,...,top_brand_84,top_sales_0,top_sales_1,top_sales_7,top_sales_9,top_sales_10,top_sales_11,top_sales_14,top_sales_17,top_sales_31,top_sales_32,top_sales_34,top_sales_39,top_sales_41,top_sales_84,top_sales_133,Metro_dist,Metro_num,mon_work_time,tue_work_time,wed_work_time,thu_work_time,fri_work_time,sat_work_time,sun_work_time,weekdays_work_time,weekend_work_time,all_days_work_time,is_work_time_24_7,is_work_time_almost_24_7,uniq_sku_sum,uniq_sku_sum_summer,uniq_sku_sum_autumn,uniq_sku_sum_winter,uniq_sku_sum_spring,uniq_sku_mean,uniq_sku_mean_summer,uniq_sku_mean_autumn,uniq_sku_mean_winter,uniq_sku_mean_spring
0,WRUE00003190,NaN,0,"Брянская обл, Брянск г, Советский округ","Брянская область, Брянск",800280,NaN,NaN,NaN,обл,г,NaN,ул,1.0,1.0,NaN,NaN,Понедельник: с 8:00 до 18:00,Вторник: с 8:00 до 18:00,Среда: с 8:00 до 18:00,Четверг: с 8:00 до 18:00,Пятница: с 8:00 до 18:00,Суббота: с 9:00 до 14:00,Воскресенье: -,112158.0,253100,16.0,27.0,12.0,20.0,15.0,8.0,18.0,24.0,12.0,18.0,13.0,16.0,26.0,19.0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,10.0,10.0,10.0,10.0,10.0,5.0,0.0,50.0,5.0,55.0,0.0,0.0,202.0,43.0,54.0,47.0,58.0,16.833333,14.333333,18.0,15.666667,19.333333
1,WRUE00011494,2052648.0,1,"Челябинская обл, Челябинск г, Курчатовский округ","Челябинская область, Челябинск",800107,NaN,NaN,NaN,обл,г,NaN,пр-кт,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,227841.0,385560,50.0,86.0,37.0,4.0,1.0,0.0,15.0,17.0,19.0,9.0,3.0,3.0,3.0,3.0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.0,5.0,51.0,15.0,9.0,6.666667,1.666667,17.0,5.000000,3.000000
2,WRUE00016734,NaN,0,"Краснодарский край, Городской округ Новороссий...","Краснодарский край, Новороссийск",800306,NaN,438.00,NaN,край,г,NaN,ул,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,147609.0,398397,47.0,31.0,31.0,17.0,16.0,15.0,0.0,0.0,0.0,0.0,0.0,8.0,21.0,25.0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,118.0,48.0,0.0,8.0,62.0,9.833333,16.000000,0.0,2.666667,20.666667
3,WRUE00016882,NaN,0,"Новгородская обл, Великий Новгород г, Великий ...","Новгородская область, Великий Новгород",800164,NaN,NaN,NaN,обл,г,NaN,пр-кт,1.0,1.0,NaN,NaN,Понедельник: с 8:00 до 19:00,Вторник: с 8:00 до 19:00,Среда: с 8:00 до 19:00,Четверг: с 8:00 до 19:00,Пятница: с 8:00 до 19:00,Суббота: -,Воскресенье: -,224297.0,441932,37.0,39.0,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,11.0,11.0,11.0,11.0,11.0,11.0,0.0,55.0,11.0,66.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,WRUE00020818,NaN,0,"Ростовская обл, Ростов-на-Дону г, Октябрьский ...","Ростовская область, Ростов-на-Дону, Октябрьски...",800200,NaN,NaN,NaN,обл,г,NaN,ул,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,167935.0,318782,0.0,21.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## *SELL_* (отрицательные никак не обрабатываются)

In [ ]:
pharm_df[sell_col][(pharm_df[sell_col[0]] < 0) 
                   | (pharm_df[sell_col[1]] < 0)
                   | (pharm_df[sell_col[2]] < 0)
                   | (pharm_df[sell_col[3]] < 0)
                   | (pharm_df[sell_col[4]] < 0)
                   | (pharm_df[sell_col[5]] < 0)
                   | (pharm_df[sell_col[6]] < 0)
                   | (pharm_df[sell_col[7]] < 0)
                   | (pharm_df[sell_col[8]] < 0)
                   | (pharm_df[sell_col[9]] < 0)
                   | (pharm_df[sell_col[10]] < 0)
                   | (pharm_df[sell_col[11]] < 0)].head()

,SELL_Jun19,SELL_Jul19,SELL_Aug19,SELL_Sep19,SELL_Oct19,SELL_Nov19,SELL_Dec19,SELL_Jan20,SELL_Feb20,SELL_Mar20,SELL_Apr20,SELL_May20
337,-2.0,0.0,0.0,68.0,42.0,39.0,91.0,70.0,92.0,103.0,74.0,16.0
1027,87.0,0.0,-1.0,308.0,329.0,255.0,513.0,520.0,230.0,261.0,293.0,0.0
1380,271.0,149.0,155.0,174.0,171.0,170.0,-232.0,-165.0,201.0,194.0,222.0,166.0
1418,99.0,0.0,-3.0,95.0,86.0,74.0,177.0,163.0,92.0,42.0,76.0,81.0
1538,95.0,13.0,-1.0,52.0,52.0,70.0,54.0,54.0,20.0,164.0,44.0,47.0


In [ ]:
# сумма продаж за год
sell_sum = pharm_df[sell_col].sum(axis=1, skipna=False)

In [ ]:
# сумма продаж по сезонам (лето, осень, зима, весна)
sell_sum_summer = pharm_df[sell_col[0:3]].sum(axis=1, skipna=False)  # лето
sell_sum_autumn = pharm_df[sell_col[3:6]].sum(axis=1, skipna=False)  # осень
sell_sum_winter = pharm_df[sell_col[6:9]].sum(axis=1, skipna=False)  # зима
sell_sum_spring = pharm_df[sell_col[9:12]].sum(axis=1, skipna=False)  # весна

In [ ]:
# среднее по продажам за год
sell_mean = pharm_df[sell_col].mean(axis=1, skipna=False)

In [ ]:
# среднее по продажам по сезонам (лето, осень, зима, весна)
sell_mean_summer = pharm_df[sell_col[0:3]].mean(axis=1, skipna=False)  # лето
sell_mean_autumn = pharm_df[sell_col[3:6]].mean(axis=1, skipna=False)  # осень
sell_mean_winter = pharm_df[sell_col[6:9]].mean(axis=1, skipna=False)  # зима
sell_mean_spring = pharm_df[sell_col[9:12]].mean(axis=1, skipna=False)  # весна

In [ ]:
# добавление признаков в таблицу
pharm_df["sell_sum"] = sell_sum
pharm_df["sell_sum_summer"] = sell_sum_summer
pharm_df["sell_sum_autumn"] = sell_sum_autumn
pharm_df["sell_sum_winter"] = sell_sum_winter
pharm_df["sell_sum_spring"] = sell_sum_spring

pharm_df["sell_mean"] = sell_mean
pharm_df["sell_mean_summer"] = sell_mean_summer
pharm_df["sell_mean_autumn"] = sell_mean_autumn
pharm_df["sell_mean_winter"] = sell_mean_winter
pharm_df["sell_mean_spring"] = sell_mean_spring

## *UNIQ_SKU_ + SELL_* (отрицательные никак не обрабатываются)

In [ ]:
# среднее кол-во продаж одного товара за год
avg_sell_sku = sell_sum / uniq_sku_sum

In [ ]:
# среднее кол-во продаж одного товара по сезонам (лето, осень, зима, весна)
avg_sell_sku_summer = sell_sum_summer / uniq_sku_sum_summer
avg_sell_sku_autumn = sell_sum_autumn / uniq_sku_sum_autumn
avg_sell_sku_winter = sell_sum_winter / uniq_sku_sum_winter
avg_sell_sku_spring = sell_sum_spring / uniq_sku_sum_spring

In [ ]:
# добавление признаков в таблицу
pharm_df["avg_sell_sku"] = avg_sell_sku
pharm_df["avg_sell_sku_summer"] = avg_sell_sku_summer
pharm_df["avg_sell_sku_autumn"] = avg_sell_sku_autumn
pharm_df["avg_sell_sku_winter"] = avg_sell_sku_winter
pharm_df["avg_sell_sku_spring"] = avg_sell_sku_spring

## *SELL_* (обнуление отрицательных продаж)

In [ ]:
# создадим новую таблицу со столбцами sell_col
pharm_df_ = pharm_df[sell_col]

In [ ]:
# обнулим все отрицательные продажи
pharm_df_[pharm_df_ < 0] = 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:2986: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


In [ ]:
# сумма продаж за год
sell_sum_0 = pharm_df_[sell_col].sum(axis=1, skipna=False)

In [ ]:
# сумма продаж по сезонам (лето, осень, зима, весна)
sell_sum_summer_0 = pharm_df_[sell_col[0:3]].sum(axis=1, skipna=False)  # лето
sell_sum_autumn_0 = pharm_df_[sell_col[3:6]].sum(axis=1, skipna=False)  # осень
sell_sum_winter_0 = pharm_df_[sell_col[6:9]].sum(axis=1, skipna=False)  # зима
sell_sum_spring_0 = pharm_df_[sell_col[9:12]].sum(axis=1, skipna=False)  # весна

In [ ]:
# среднее по продажам за год
sell_mean_0 = pharm_df_[sell_col].mean(axis=1, skipna=False)

In [ ]:
# среднее по продажам по сезонам (лето, осень, зима, весна)
sell_mean_summer_0 = pharm_df_[sell_col[0:3]].mean(axis=1, skipna=False)  # лето
sell_mean_autumn_0 = pharm_df_[sell_col[3:6]].mean(axis=1, skipna=False)  # осень
sell_mean_winter_0 = pharm_df_[sell_col[6:9]].mean(axis=1, skipna=False)  # зима
sell_mean_spring_0 = pharm_df_[sell_col[9:12]].mean(axis=1, skipna=False)  # весна

In [ ]:
# добавление признаков в таблицу
pharm_df["sell_sum_0"] = sell_sum_0
pharm_df["sell_sum_summer_0"] = sell_sum_summer_0
pharm_df["sell_sum_autumn_0"] = sell_sum_autumn_0
pharm_df["sell_sum_winter_0"] = sell_sum_winter_0
pharm_df["sell_sum_spring_0"] = sell_sum_spring_0

pharm_df["sell_mean_0"] = sell_mean_0
pharm_df["sell_mean_summer_0"] = sell_mean_summer_0
pharm_df["sell_mean_autumn_0"] = sell_mean_autumn_0
pharm_df["sell_mean_winter_0"] = sell_mean_winter_0
pharm_df["sell_mean_spring_0"] = sell_mean_spring_0

## *UNIQ_SKU_ + SELL_* (обнуление отрицательных продаж)

In [ ]:
# среднее кол-во продаж одного товара за год
avg_sell_sku_0 = sell_sum_0 / uniq_sku_sum

In [ ]:
# среднее кол-во продаж одного товара по сезонам (лето, осень, зима, весна)
avg_sell_sku_summer_0 = sell_sum_summer_0 / uniq_sku_sum_summer
avg_sell_sku_autumn_0 = sell_sum_autumn_0 / uniq_sku_sum_autumn
avg_sell_sku_winter_0 = sell_sum_winter_0 / uniq_sku_sum_winter
avg_sell_sku_spring_0 = sell_sum_spring_0 / uniq_sku_sum_spring

In [ ]:
# добавление признаков в таблицу
pharm_df["avg_sell_sku_0"] = avg_sell_sku_0
pharm_df["avg_sell_sku_summer_0"] = avg_sell_sku_summer_0
pharm_df["avg_sell_sku_autumn_0"] = avg_sell_sku_autumn_0
pharm_df["avg_sell_sku_winter_0"] = avg_sell_sku_winter_0
pharm_df["avg_sell_sku_spring_0"] = avg_sell_sku_spring_0

# **Заполнение пропусков для всего, что связано с UNIQ_SKU_ и SELL_**

## *Медианой по региону*

In [ ]:
# названия колонок с агрегированными признаками UNIQ_ и SELL_
uniq_sell_col = [col for col in pharm_df.columns if "sell_sum" in col or \
                 "sell_uniq" in col  or "uniq" in col]

In [ ]:
# новая таблица для работы
# столбцы: регионы и те столбцы с uniq_ и sell_, которые будем восстанавливать
uniq_sell_median_df = pd.DataFrame()
uniq_sell_median_df["region"] = pharm_df.OKBrickL3.map(lambda x: x.split(",")[0])
uniq_sell_col_median = [old_name + "_median" for old_name in uniq_sell_col]
uniq_sell_median_df[uniq_sell_col_median] = pharm_df[uniq_sell_col]

In [ ]:
uniq_sell_median_df.head()

,region,uniq_sku_sum_median,uniq_sku_sum_summer_median,uniq_sku_sum_autumn_median,uniq_sku_sum_winter_median,uniq_sku_sum_spring_median,uniq_sku_mean_median,uniq_sku_mean_summer_median,uniq_sku_mean_autumn_median,uniq_sku_mean_winter_median,uniq_sku_mean_spring_median,sell_sum_median,sell_sum_summer_median,sell_sum_autumn_median,sell_sum_winter_median,sell_sum_spring_median,sell_sum_0_median,sell_sum_summer_0_median,sell_sum_autumn_0_median,sell_sum_winter_0_median,sell_sum_spring_0_median
0,Брянская обл,202.0,43.0,54.0,47.0,58.0,16.833333,14.333333,18.0,15.666667,19.333333,1390.0,268.0,303.0,360.0,459.0,1390.0,268.0,303.0,360.0,459.0
1,Челябинская обл,80.0,5.0,51.0,15.0,9.0,6.666667,1.666667,17.0,5.000000,3.000000,335.0,5.0,257.0,61.0,12.0,335.0,5.0,257.0,61.0,12.0
2,Краснодарский край,118.0,48.0,0.0,8.0,62.0,9.833333,16.000000,0.0,2.666667,20.666667,386.0,126.0,0.0,36.0,224.0,386.0,126.0,0.0,36.0,224.0
3,Новгородская обл,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Ростовская обл,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# уникальные регионы
region_uniq = uniq_sell_median_df["region"].unique()

In [ ]:
# заполнение null медианой по региону
for region in region_uniq:
    region_median = uniq_sell_median_df[uniq_sell_median_df.region == region]\
                                        [uniq_sell_col_median].median()
    for col, med in zip(uniq_sell_col_median, region_median):
        uniq_sell_median_df.loc[uniq_sell_median_df[uniq_sell_median_df.region == region]
                                [pd.isnull(uniq_sell_median_df[col])].index, 
                                col] = med

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


In [ ]:
# проверим для Чукосткого АО
uniq_sell_median_df[uniq_sell_median_df.region == region_uniq[77]].head()

,region,uniq_sku_sum_median,uniq_sku_sum_summer_median,uniq_sku_sum_autumn_median,uniq_sku_sum_winter_median,uniq_sku_sum_spring_median,uniq_sku_mean_median,uniq_sku_mean_summer_median,uniq_sku_mean_autumn_median,uniq_sku_mean_winter_median,uniq_sku_mean_spring_median,sell_sum_median,sell_sum_summer_median,sell_sum_autumn_median,sell_sum_winter_median,sell_sum_spring_median,sell_sum_0_median,sell_sum_summer_0_median,sell_sum_autumn_0_median,sell_sum_winter_0_median,sell_sum_spring_0_median
7511,Чукотский АО,127.0,56.0,20.0,8.0,43.0,10.583333,18.666667,6.666667,2.666667,14.333333,805.0,236.0,200.0,74.0,295.0,805.0,236.0,200.0,74.0,295.0
10531,Чукотский АО,24.0,0.0,3.0,12.0,9.0,2.000000,0.000000,1.000000,4.000000,3.000000,436.0,0.0,50.0,283.0,103.0,436.0,0.0,50.0,283.0,103.0
10690,Чукотский АО,62.0,39.0,7.0,2.0,14.0,5.166667,13.000000,2.333333,0.666667,4.666667,550.0,263.0,65.0,40.0,182.0,550.0,263.0,65.0,40.0,182.0
17073,Чукотский АО,74.5,39.5,15.5,10.5,15.5,6.208333,13.166667,5.166667,3.500000,5.166667,1191.5,315.0,222.0,167.5,238.5,1191.5,315.0,222.0,167.5,238.5
17074,Чукотский АО,80.0,40.0,8.0,13.0,19.0,6.666667,13.333333,2.666667,4.333333,6.333333,1578.0,659.0,126.0,166.0,627.0,1578.0,659.0,126.0,166.0,627.0


In [ ]:
# по заполненным пропускам найдем теперь признаки sell / uniq_sell
suf = ["", "_summer", "_autumn", "_winter", "_spring"]
for s in suf:
  # без обнуления продаж
  uniq_sell_median_df["avg_sell_sku" + s + "_median"] = \
      uniq_sell_median_df["sell_sum" + s + "_median"] / \
      uniq_sell_median_df["uniq_sku_sum" + s + "_median"]
  # с обнулением продаж
  uniq_sell_median_df["avg_sell_sku" + s + "_0_median"] = \
      uniq_sell_median_df["sell_sum" + s + "_0_median"] / \
      uniq_sell_median_df["uniq_sku_sum" + s + "_median"]

In [ ]:
# добавим в исходную таблицу uniq_sell_median_df
pharm_df = pd.concat([pharm_df, uniq_sell_median_df], axis=1)

## *Медианой по региону и BrandCode*

Если глянуть выборочно, то у BrandCode, которые не в топе, колебания меньше. Вероятно, это даст чуть более точный результат для данного способа

In [ ]:
# новая таблица для работы
# столбцы: регионы, BrandCodes и те столбцы с uniq_ и sell_, которые будем восстанавливать
# uniq_sell_col - см. выше
uniq_sell_median2_df = pd.DataFrame()
uniq_sell_median2_df["region"] = pharm_df.OKBrickL3.map(lambda x: x.split(",")[0])
uniq_sell_median2_df["BrandCodes"] = pharm_df.BrandCodes
uniq_sell_col_median2 = [old_name + "_median2" for old_name in uniq_sell_col]
uniq_sell_median2_df[uniq_sell_col_median2] = pharm_df[uniq_sell_col]

In [ ]:
uniq_sell_median2_df.head()

In [ ]:
# заполнение null медианой по региону и BrandCodes
for region in region_uniq:
    # уникальные BrandCodes для данного региона
    bc_uniq = uniq_sell_median2_df[uniq_sell_median2_df.region == region].BrandCodes.unique()
    
    # пробежимся по всем BrandCodes
    for bc in bc_uniq:  
        region_median = uniq_sell_median2_df[(uniq_sell_median2_df.region == region) \
                                             & (uniq_sell_median2_df.BrandCodes == bc)] \
                                            [uniq_sell_col_median2].median()
        for col, med in zip(uniq_sell_col_median2, region_median):
            uniq_sell_median2_df.loc[uniq_sell_median2_df[(uniq_sell_median2_df.region == region) \
                                                          & (uniq_sell_median2_df.BrandCodes == bc) \
                                                          & (pd.isnull(uniq_sell_median2_df[col]))].index, \
                                                          col] = med

In [ ]:
# глянем для Чукотки
uniq_sell_median2_df[(uniq_sell_median2_df.region == region_uniq[77]) \
                    & (uniq_sell_median2_df.BrandCodes == 0)]

In [ ]:
# по заполненным пропускам найдем теперь признаки sell / uniq_sell
suf = ["", "_summer", "_autumn", "_winter", "_spring"]
for s in suf:
  # без обнуления продаж
  uniq_sell_median2_df["avg_sell_sku" + s + "_median2"] = \
      uniq_sell_median2_df["sell_sum" + s + "_median2"] / \
      uniq_sell_median2_df["uniq_sku_sum" + s + "_median2"]
  # с обнулением продаж
  uniq_sell_median2_df["avg_sell_sku" + s + "_0_median2"] = \
      uniq_sell_median2_df["sell_sum" + s + "_0_median2"] / \
      uniq_sell_median2_df["uniq_sku_sum" + s + "_median2"]

In [ ]:
# добавим в исходную таблицу uniq_sell_median_df
pharm_df = pd.concat([pharm_df, uniq_sell_median2_df], axis=1)

#Генерация дополнительных признаков

In [ ]:
region = pharm_df.groupby("region")["region"].agg("count")
pharm_df["num_pharm"] = pharm_df.region.map(lambda x: region.loc[x])

In [ ]:
sum(pharm_df.loc[pharm_df["region"] == region.index[0]].groupby("population").agg("count").index)

437984.0

In [ ]:
dict_pop = {i:0 for i in region.index}
for i in pharm_df.groupby(["region","population"])["population"].agg(["count"]).index:
  dict_pop[i[0]] = dict_pop.get(i[0]) + i[1]

In [ ]:
pharm_df["region_popul"] = pharm_df.region.map(lambda x: dict_pop[x])

In [ ]:
pharm_df["soul_on_pharm"] =pharm_df["region_popul"] / pharm_df["num_pharm"]

In [ ]:
pens_df = pd.read_excel("/content/drive/My Drive/SHAD/Pharm/22720000300200200001_Общая_численность_пенсионеров.xlsx",header=1)
pens_df.drop(["Unnamed: 1"], axis =1, inplace=True)
pens_df.columns = ["Region", "JAN2019", "JAN2020"]
pens_df.Region = pens_df.Region.map(lambda x: re.sub("область", "обл", x))
debcred_df = pd.read_excel("/content/drive/My Drive/SHAD/Pharm/dohodi_rashodi.xls",
                           header =2)
debcred_df.drop(["Unnamed: 0"], axis=1,inplace=True)
debcred_df.columns = ["Region","JAN2019"]
debcred_df.Region = debcred_df.Region.map(lambda x: x.strip()).map(lambda x: re.sub("область", "обл", x))
bascket_df = pd.read_excel("/content/drive/My Drive/SHAD/Pharm/minimal_basket.xls",header =3)
bascket_df.columns = ["Region",
                      "JAN2019", "FEB2019", "MAR2019", "APR2019",
                      "MAY2019", "JUN2019", "JUL2019", "AUG2019",
                      "SEP2019", "OCT2019", "NOV2019", "DEC2019",
                      "JAN2020", "FEB2020", "MAR2020", "APR2020",
                      "MAY2020",
                      ]

In [ ]:
pharm_df.merge(pens_df,
               left_on="region",
               right_on="Region")\
        .merge(debcred_df,
               left_on="Region",
               right_on="Region",
               how="left")

,OneKeyID,GlobalCode,BrandCodes,OKBrickL3,AuditBrickL4,BrickCode,SquareMeterPrice,FlatArea,BeltwayHit,RegionType,CityType,SettlementType,StreetType,Cash,Card,Internet,Bank,Mon,Tue,Wed,Thu,Fri,Sat,Sun,population,VRP,NumHos,NumAmb,NumPhc,UNIQ_SKU_Jun19,UNIQ_SKU_Jul19,UNIQ_SKU_Aug19,UNIQ_SKU_Sep19,UNIQ_SKU_Oct19,UNIQ_SKU_Nov19,UNIQ_SKU_Dec19,UNIQ_SKU_Jan20,UNIQ_SKU_Feb20,UNIQ_SKU_Mar20,UNIQ_SKU_Apr20,...,avg_sell_sku_winter_0,avg_sell_sku_spring_0,region,uniq_sku_sum_median,uniq_sku_sum_summer_median,uniq_sku_sum_autumn_median,uniq_sku_sum_winter_median,uniq_sku_sum_spring_median,uniq_sku_mean_median,uniq_sku_mean_summer_median,uniq_sku_mean_autumn_median,uniq_sku_mean_winter_median,uniq_sku_mean_spring_median,sell_sum_median,sell_sum_summer_median,sell_sum_autumn_median,sell_sum_winter_median,sell_sum_spring_median,sell_sum_0_median,sell_sum_summer_0_median,sell_sum_autumn_0_median,sell_sum_winter_0_median,sell_sum_spring_0_median,avg_sell_sku_median,avg_sell_sku_0_median,avg_sell_sku_summer_median,avg_sell_sku_summer_0_median,avg_sell_sku_autumn_median,avg_sell_sku_autumn_0_median,avg_sell_sku_winter_median,avg_sell_sku_winter_0_median,avg_sell_sku_spring_median,avg_sell_sku_spring_0_median,num_pharm,region_popul,soul_on_pharm,Region,JAN2019_x,JAN2020,JAN2019_y
0,WRUE00003190,NaN,0,"Брянская обл, Брянск г, Советский округ","Брянская область, Брянск",800280,NaN,NaN,NaN,обл,г,NaN,ул,1.0,1.0,NaN,NaN,Понедельник: с 8:00 до 18:00,Вторник: с 8:00 до 18:00,Среда: с 8:00 до 18:00,Четверг: с 8:00 до 18:00,Пятница: с 8:00 до 18:00,Суббота: с 9:00 до 14:00,Воскресенье: -,112158.0,253100,16.0,27.0,12.0,20.0,15.0,8.0,18.0,24.0,12.0,18.0,13.0,16.0,26.0,19.0,...,7.659574,7.913793,Брянская обл,202.0,43.0,54.0,47.0,58.0,16.833333,14.333333,18.000000,15.666667,19.333333,1390.000000,268.0000,303.0000,360.000000,459.00000,1390.000000,268.0000,303.0000,360.000000,459.00000,6.881188,6.881188,6.232558,6.232558,5.611111,5.611111,7.659574,7.659574,7.913793,7.913793,718,1006392.0,1401.660167,Брянская обл,439.99,437.00,407109521
1,WRUF00000566,2048986.0,1,"Брянская обл, Брянск г, Советский округ","Брянская область, Брянск",800280,NaN,NaN,NaN,обл,г,NaN,ул,1.0,1.0,NaN,NaN,Понедельник: с 8:00 до 21:00,Вторник: с 8:00 до 21:00,Среда: с 8:00 до 21:00,Четверг: с 8:00 до 21:00,Пятница: с 8:00 до 21:00,Суббота: с 8:00 до 21:00,Воскресенье: с 8:00 до 21:00,112158.0,253100,2.0,15.0,29.0,113.0,97.0,25.0,97.0,103.0,93.0,90.0,99.0,99.0,103.0,79.0,...,4.216146,4.362595,Брянская обл,1078.0,235.0,293.0,288.0,262.0,89.833333,78.333333,97.666667,96.000000,87.333333,4147.916600,817.0000,973.6666,1214.250000,1143.00000,4147.916600,817.0000,973.6666,1214.250000,1143.00000,3.847789,3.847789,3.476596,3.476596,3.323094,3.323094,4.216146,4.216146,4.362595,4.362595,718,1006392.0,1401.660167,Брянская обл,439.99,437.00,407109521
2,WRUF00000721,NaN,71,"Брянская обл, Брянск г, Советский округ","Брянская область, Брянск",800280,32143.0,24.70,NaN,обл,г,NaN,ул,1.0,1.0,NaN,1.0,Понедельник: с 8:00 до 21:00,Вторник: с 8:00 до 21:00,Среда: с 8:00 до 21:00,Четверг: с 8:00 до 21:00,Пятница: с 8:00 до 21:00,Суббота: с 8:00 до 21:00,Воскресенье: с 8:00 до 21:00,112158.0,253100,21.0,46.0,49.0,76.0,32.0,34.0,11.0,24.0,45.0,30.0,41.0,32.0,37.0,37.0,...,9.291262,4.830357,Брянская обл,437.0,142.0,80.0,103.0,112.0,36.416667,47.333333,26.666667,34.333333,37.333333,2529.000000,723.0000,308.0000,957.000000,541.00000,2529.000000,723.0000,308.0000,957.000000,541.00000,5.787185,5.787185,5.091549,5.091549,3.850000,3.850000,9.291262,9.291262,4.830357,4.830357,718,1006392.0,1401.660167,Брянская обл,439.99,437.00,407109521
3,WRUF00000722,2048976.0,1,"Брянская обл, Брянск г, Володарский округ","Брянская область, Брянск",800280,NaN,19.00,NaN,обл,г,NaN,ул,1.0,1.0,NaN,NaN,Понедельник: с 8:00 до 21:00,Вторник: с 8:00 до 21:00,Среда: с 8:00 до 21:00,Четверг: с 8:00 до 21:00,Пятница: с 8:00 до 21:00,Суббота: с 8:00 до 21:00,Воскресенье: с 9:00 до 21:00,69685.0,253100,16.0,2.0,14.0,140.0,135.0,48.0,126.0,127.0,123.0,1